# RL for LLMs

## Background: REINFORCE

Loss $l(f_\theta(x)) = \left(10 - \lvert f_\theta(x)\rvert\right)^2$ is non-differentiable, $x$ is a sentence, $\theta$ are model weights. Objective:

$$\min_\theta l(f_\theta(x))$$

We can't optimize the objective directly, because $l$ is non-differentiable.
Instead, we get the expectation of the loss by sampling from a distribution which we _can_ optimize.

Now we seek another objective that's similar

\begin{align}
&\min_\mu\int l(f_\theta(x))\ p(\theta;\mu)\ d\theta\\
=&E_{p(\theta;\mu)} [l(f_\theta(x))]\\
\approx&\frac{1}{N} \sum_{n=1}^Nl(f_{\theta_n}(x)) \quad \text{with}\quad\theta_n\sim p(\theta;\mu) \quad \text{(MC estimation)}
\end{align}

$p(\theta;\mu)$ could be something like $\mathcal{N}(\mu,1)$

To optimize for $\mu$:

\begin{align}
&\nabla_\mu\int l(f_\theta(x))\ p(\theta;\mu)\ d\theta\\
=&\int \underbrace{l(f_\theta(x))}_\text{net non diff.}\ \nabla_\mu p(\theta;\mu)\ d\theta\\
\end{align}

---

REINFORCE gradient estimate / score function estimate:

\begin{align}
\nabla_x\ln(x) =& \frac{1}{x}\nabla_xx\\
\underbrace{\nabla_\mu\ln(p(\theta;\mu))}_{\text{score function}} =& \frac{1}{p(\theta;\mu)}\nabla_\mu p(\theta;\mu)\quad\vert \cdot p(\theta;\mu)\\
\nabla_\mu\ln(p(\theta;\mu)) \cdot p(\theta;\mu) =& \nabla_\mu p(\theta;\mu)\\
\end{align}

---

\begin{align}
&\int l(f_\theta(x))\ \nabla_\mu p(\theta;\mu)\ d\theta\\
=&\int l(f_\theta(x))\ \nabla_\mu\ln(p(\theta;\mu)) \cdot p(\theta;\mu)\ d\theta\\
\approx&\frac{1}{n}\sum_i^n l(f_{\theta_i}(x))\ \nabla_\mu\ln(p(\theta_i;\mu))\quad\theta_i\sim p(\theta;\mu)\\
=&
\end{align}

Assume $p = \mathcal{N}(\theta;\mu,I)$ (we go through the 1D case here):

\begin{align*}
\nabla_\mu\ln(p(\theta_i;\mu)) =& \nabla_\mu\ln\left( \frac{1}{\sigma \sqrt{2\pi}} \exp\left( -\frac{(\theta - \mu)^2}{2\sigma^2} \right)\right)\\
=& \underbrace{\nabla_\mu \ln\left( \frac{1}{\sigma \sqrt{2\pi}} \right)}_{=0} - \nabla_\mu \frac{(\theta - \mu)^2}{2\sigma^2}\\
=& -\frac{1}{2\sigma^2} 2 (\theta-\mu) \cdot (-1)\\
=& (\theta-\mu)
\end{align*}

Does REINFORCE work for LLMs? Concern is that sampling from $p(\theta_i;\mu)$ may not work for a $\theta$ which is high dim (e.g., 70M or 175B in the real world).

Gradient update with (3)

\begin{align}
\mu^{(t+1)} \gets& \mu^{(t)} - \alpha \underbrace{\frac{1}{n}\sum_i^n l(f_{\theta_i}(x))\ \nabla_\mu\ln(p(\theta_i;\mu))\quad\theta_i\sim p(\theta;\mu)}_{(3)} \\
\gets& \mu^{(t)} - \alpha \frac{1}{n}\sum_i^n l(f_{\theta_i}(x))\ (\theta_i - \mu)\quad\theta_i\sim p(\theta;\mu)
\end{align}

In [5]:
from sklearn.datasets import load_iris
import torch

In [18]:
x, y = load_iris(return_X_y=True)
x, y = torch.tensor(x).float(), torch.tensor(y)
x.shape, y.shape

(torch.Size([150, 4]), torch.Size([150]))

In [19]:
torch.manual_seed(42);

\begin{align}
\min_w \quad \int \operatorname{acc}(y, y_\text{true}) p(y;w)\ dy\\
\min_w \quad \sum_{i} \operatorname{acc}(y_i, y_\text{true}) p(y(x);w)_i\\
\end{align}

We could use (2) if we didn't want to sample. For a discrete RV with just four possible outcomes as we have it here we could loop, for an LLM the # of outcomes is much larger. Below we do the case where we sample from the discrete RM.

\begin{align}
\nabla_w \int \operatorname{acc}(y, y_\text{true}) p(y;w)\ dy =& \int \operatorname{acc}(y, y_\text{true})\ \nabla_w p(y;w)\ dy \\
=& \int \operatorname{acc}(y, y_\text{true})\ \nabla_w\ln(p(y;w)) \cdot p(y;w)\ dy \quad \vert \quad \text{for a given }x\\
\approx& \frac{1}{N}\sum_{i=1}^N \operatorname{acc}(y_n(x), y_\text{true})\ \nabla_w\ln(p(y_n(x);w)) \quad \text{with} \quad y_n\sim p(y(x);w)
\end{align}

\begin{align}
&\nabla_w \quad \sum_{i} \operatorname{acc}(y_i, y_\text{true})\ p(y(x);w)_i\\
=&\sum_{i} \operatorname{acc}(y_i, y_\text{true})\ \nabla_w p(y(x);w)_i\\
=&\sum_{i} \operatorname{acc}(y_i, y_\text{true})\ \nabla_w \log p(y(x);w)_i \cdot p(y(x);w)\\
=&\sum_{i=1}^N \operatorname{acc}(y_i, y_\text{true})\ \nabla_w \underbrace{\log p(y(x);w)_i}_{\text{log\_probs[drawn\_class]}} \quad y_i \sim p(y(x);w)\\
\end{align}

In [175]:
net = torch.nn.Sequential(
    torch.nn.Linear(in_features=x.shape[1], out_features=4),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=4, out_features=3),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=3, out_features=4),
)

In [176]:
opt = torch.optim.Adam(net.parameters(), lr=1e-3)

rewards = []

num_epochs = 10000
num_mc_samples = 1
for i in range(num_epochs):
    logits = net(x)
    probs = torch.softmax(logits, dim=-1)
    drawn_class = torch.multinomial(probs, num_samples=1).squeeze()
    reward = (drawn_class == y).float()
    log_probs = torch.log_softmax(logits, dim=-1)
    selected_log_probs = log_probs.index_select(dim=-1, index=drawn_class)

    net.zero_grad()
    loss = 1 / num_mc_samples * (-reward * selected_log_probs).mean()
    loss.backward()
    opt.step()

    rewards.append(reward.mean())

In [177]:
rewards = torch.asarray(rewards)

In [179]:
rewards[:10].mean(), rewards[-50:].mean()

(tensor(0.3107), tensor(0.3371))

## Background: PPO

Importance sampling: 

\begin{align}
&\int f(x)p(x)\ dx = E_p[f(x)]\\
=&\int f(x)p(x) \cdot 1\ dx\\
=&\int f(x)p(x) \cdot \frac{q(x)}{q(x)}\ dx\\
=&\int f(x)q(x) \cdot \frac{p(x)}{q(x)}\ dx = E_q\left[f(x)\frac{p(x)}{q(x)}\right]\\
\end{align}

Rewrite after looking into `grpo_loss` in [this file](https://github.com/aburkov/theLMbook/blob/main/GRPO.py).

| Symbol | Meaning |
| ---:| --- |
| $\phi$ | model weights |
| $x,y$ | model input (prompt) and sampled outputs |
| $r(x,y)$ | reward |
| $\pi_\phi^\text{RL}$ | policy (model) |

What we want to find / our objective:

\begin{align}
&\displaystyle \nabla_\phi E_{\pi_\phi^\text{RL}(x,y)}\left[r(x,y)\right]\\
=&\nabla_\phi\int r(x,y)\ \pi_\phi^\text{RL}(x,y)\ dxdy\\
=&\int r(x,y)\ \nabla_\phi \pi_\phi^\text{RL}(x,y)\ dxdy\\
=&\int r(x,y)\ \nabla_\phi \ln \pi_\phi^\text{RL}(x,y) \cdot \pi_\phi^\text{RL}(x,y)\ dxdy\\
\approx&\frac{1}{N}\sum_n^N r(x,y)\  \nabla_\phi \ln \pi_\phi^\text{RL}(x_n,y_n) & \text{with } (x_n,y_n) \sim \pi_\phi^\text{RL}(x,y)\\
\end{align}